In [10]:
import numpy as np
import pandas as pd
from scipy import sparse
import re
import xgboost ; import lightgbm

ImportError: No module named 'xgboost'

In [2]:
train = pd.read_csv('train.csv')
test = pd.read_csv('test.csv')
song = pd.read_csv('songs.csv')
members =  pd.read_csv('members.csv')

In [3]:
train = train.merge(song,on= 'song_id',how = 'left')
test = test.merge(song,on = 'song_id',how = 'left')

In [7]:
#user_bias, user_freq, song_bias, song_freq
tuf = train.groupby('msno')['target'].agg(['sum','count']) #Laplacian smoothing
tuf = tuf.reset_index()
tuf.columns.values[1:3] = ['user_sum','user_count']
train = train.merge(tuf,on = 'msno',how = 'left')
train['user_freq'] = (train.user_sum-train.target+1)/(train.user_count+1)
train['user_count'] = train['user_count'] - 1
train = train.drop('user_sum',1)

#prepare test's user bias
tuf['user_freq'] = (tuf.user_sum+1)/(tuf.user_count+2)
test = test.merge(tuf,on = 'msno',how = 'left')
test.loc[pd.isnull(test.user_count),'user_count'] = 0
test.loc[pd.isnull(test.user_freq),'user_freq'] = 0.5
test = test.drop('user_sum',1)

#prepare songs' bias
tuf = train.groupby('song_id')['target'].agg(['sum','count']) #Laplacian smoothing
tuf = tuf.reset_index()
tuf.columns.values[1:3] = ['song_sum','song_count']
train = train.merge(tuf,on = 'song_id',how = 'left')
train['song_freq'] = (train.song_sum-train.target+1)/(train.song_count+1)
train['song_count'] = train['song_count'] -1
train = train.drop('song_sum',1)

#prepare test's songs' bias
tuf['song_freq'] = (tuf.song_sum+1)/(tuf.song_count+2)
test = test.merge(tuf,on = 'song_id',how = 'left')
test.loc[pd.isnull(test.song_count),'song_count'] = 0
test.loc[pd.isnull(test.song_freq),'song_freq'] = 0.5
test = test.drop('song_sum',1)

Artist Preparation: 
1. find the intersection of artist between the train and test
2. treat the compound artist seperately from single artist
3. if a single artist from test set hasn't appeared before, we can do nothing, however, if a compound artist has appeared, we can possibly estimate it from the single artist's contribution
4. add a dummy variable artist_estimated to show

In [5]:
def split(x):
    l = re.split("[+|&/]",x)
    l = [i.strip() for i in list(l)]
    return l

In [8]:
train.artist_name = train.artist_name.fillna("no_rec")
test.artist_name = test.artist_name.fillna("no_rec")

In [9]:
train_art = set()
for name in train.artist_name.unique():
    train_art.add(name)
    for sub in split(name):
        train_art.add(sub.strip())

In [10]:
test_art_diff = set()
for name in test.artist_name.unique():
    indic = [sub.strip() in train_art for sub in split(name)]
    if not any(indic):
        test_art_diff.add(name)
#19%的artist从来没有出现过

In [11]:
tt_csame = set()
for name in test.artist_name.unique():
     if len(split(name)) > 1 and name in train_art:
            tt_csame.add(name)

In [12]:
tt_single = set()
for name in train.artist_name.unique():
    if len(split(name)) == 1:
        tt_single.add(name)

In [13]:
tt_cdump = set()
for name in train.artist_name.unique():
    if len(split(name)) > 1 and not name in tt_csame:
        tt_cdump.add(name)

In [ ]:
artdict = dict(zip(list(train_art),range(len(train_art))))
userdict = dict(zip(train.msno.unique(),range(train.msno.nunique())))
v = train.groupby(['msno','artist_name'])['target'].agg(['sum','count'])
v = v.reset_index()
v.columns.values[2:] = ['uasum','uacount']
uamat = sparse.lil_matrix((len(userdict),2*len(artdict)))
v['mind'] = v.msno.apply(lambda x: userdict[x])
v['amind'] = v.artist_name.apply(lambda x: artdict[x])
for i in v.index:
    uamat[v.mind[i],2*v.amind[i]] = v.uasum[i]
    uamat[v.mind[i],2*v.amind[i]+1] = v.uacount[i]

In [ ]:
disdict = {}
for key, ind in artdict.items():
    l = [artdict[sub] for sub in split(key) if not sub in tt_single]
    if len(l) > 0:
        disdict[key] = (l,ind)
uamat = sparse.csc_matrix(uamat)
for key, ind in disdict.items():
    for sub in ind[0]:
        uamat[:,2*sub] += uamat[:,2*ind[1]]
        uamat[:,2*sub+1] += uamat[:,2*ind[1]+1]

In [32]:
#prepare train artist feature
oart = train.groupby('artist_name')['target'].agg(['sum','count'])
oart = oart.reset_index()
oart.columns.values[1:3] = ['art_sum','art_count']
train = train.merge(oart,on='artist_name',how = 'left')
train['art_freq'] = (train.art_sum-train.target+1)/(train.art_count+1)
traindump = train[train.artist_name.isin(list(tt_cdump))]
train['is_art_dump'] = 0
uasum = uamat.sum(axis = 0)
for name,group in traindump.groupby("artist_name"):
    subind = np.array([artdict[sub.strip()] for sub in split(name)])
    gasum = uasum[0,2*subind].sum()/len(subind)
    gacount = uasum[0,2*subind+1].sum()/len(subind)
    for ind in group.index:
        train.set_value(ind,'art_sum',gasum)
        train.set_value(ind,'art_count',gacount)
        train.set_value(ind,'art_freq',gasum/gacount)
        train.set_value(ind,'is_art_dump',1)
train = train.drop('art_sum',1)
train.art_count = train.art_count -1 

In [33]:
#prepare test artist feature
oart['art_freq'] = oart.art_sum/oart.art_count
test = test.merge(oart,on='artist_name',how = 'left')
test['is_art_dump'] = 0
testnan = test[pd.isnull(test.art_sum)]
for name, group in testnan.groupby("artist_name"):
    subind = np.array([artdict[sub.strip()] for sub in split(name) if sub.strip() in train_art])
    if subind.size == 0:
        for ind in group.index:
            test.set_value(ind,'art_count',0)
            test.set_value(ind,'art_freq',0.5)
    else:
        gasum = uasum[0,2*subind].sum()/len(subind)
        gacount = uasum[0,2*subind+1].sum()/len(subind)
        for ind in group.index:
            test.set_value(ind,'art_count',gacount)
            test.set_value(ind,'art_freq',gasum/gacount)
            test.set_value(ind,'is_art_dump',1)
test = test.drop('art_sum',1)

/Users/yanfangwang/anaconda3/envs/nlp11/lib/python3.6/site-packages/ipykernel_launcher.py:17: RuntimeWarning: invalid value encountered in double_scalars


In [72]:
#prepare train/artist feature
train = train.merge(v,on = ['msno','artist_name'],how = 'left')
train['uafreq'] = (train['uasum'] - train['target']+1)/(train['uacount']+1)
train['uacount'] = train['uacount'] -1
for name,group in train[train.artist_name.isin(list(tt_cdump))].groupby('artist_name'):
    subcolin = np.array([artdict[sub.strip()] for sub in split(name)])
    vsum = uamat[:,2*subcolin][group.mind,:].sum(axis =1)
    vcount = uamat[:,2*subcolin+1][group.mind,:].sum(axis = 1)
    gi = group.index ; gs = group.shape[0]
    for i in range(gs):
        train.set_value(gi[i],'uafreq',vsum[i]/vcount[i] if vcount[i] > 0 else 0.5)
        train.set_value(gi[i],'uacount',vcount[i]/len(subcolin))
train = train.drop(['mind','amind','uasum'],1)

In [73]:
#prepare test/artist feature
trainmsno = train.msno.unique()
v['uafreq'] = v.uasum/v.uacount
test = test.merge(v,on = ['msno','artist_name'],how = 'left')
testnan = test[pd.isnull(test.uacount)]
testnu = testnan[~testnan.msno.isin(trainmsno)]
for name,group in testnu.groupby(['msno','artist_name']):
    subcolin = np.array([artdict[sub.strip()] for sub in split(name[1]) if sub.strip() in train_art])
    indic = len(subcolin) > 0 
    overallfreq = uasum[0,2*subcolin].sum()/uasum[0,2*subcolin+1].sum() if indic else 0.5
    for ind in group.index:
        test.set_value(ind,'uacount',0)
        test.set_value(ind,'uafreq',overallfreq)
testrem = test[pd.isnull(test.uafreq)]
testrem['mind']= testrem.msno.apply(lambda x: userdict[x])
for name,group in testrem.groupby('artist_name'):
    subcolin = np.array([artdict[sub.strip()] for sub in split(name) if sub.strip() in train_art])
    if len(subcolin) > 0:
        vsum = uamat[:,2*subcolin][group.mind,:].sum(axis = 1)
        vcount = uamat[:,2*subcolin+1][group.mind,:].sum(axis =1)
        gi = group.index
        for i in range(group.shape[0]):
            test.set_value(gi[i],'uafreq',vsum[i]/vcount[i] if vcount[i] > 0 else 0.5)
            test.set_value(gi[i],'uacount',vcount[i]/len(subcolin))
    else:
        for ind in group.index:
            test.set_value(ind,'uacount',0)
            test.set_value(ind,'uafreq',group.user_freq[ind])
test = test.drop(['mind','amind','uasum'],1)

NameError: name 'train_genre' is not defined

In [74]:
for name,group in testnu.groupby(['msno','artist_name']):
    subcolin = np.array([artdict[sub.strip()] for sub in split(name[1]) if sub.strip() in train_art])
    indic = len(subcolin) > 0 
    overallfreq = uasum[0,2*subcolin].sum()/uasum[0,2*subcolin+1].sum() if indic else 0.5
    for ind in group.index:
        test.set_value(ind,'uacount',0)
        test.set_value(ind,'uafreq',overallfreq)
testrem = test[pd.isnull(test.uafreq)]
testrem['mind']= testrem.msno.apply(lambda x: userdict[x])
for name,group in testrem.groupby('artist_name'):
    subcolin = np.array([artdict[sub.strip()] for sub in split(name) if sub.strip() in train_art])
    if len(subcolin) > 0:
        vsum = uamat[:,2*subcolin][group.mind,:].sum(axis = 1)
        vcount = uamat[:,2*subcolin+1][group.mind,:].sum(axis =1)
        gi = group.index
        for i in range(group.shape[0]):
            test.set_value(gi[i],'uafreq',vsum[i]/vcount[i] if vcount[i] > 0 else 0.5)
            test.set_value(gi[i],'uacount',vcount[i]/len(subcolin))
    else:
        for ind in group.index:
            test.set_value(ind,'uacount',0)
            test.set_value(ind,'uafreq',group.user_freq[ind])
test = test.drop(['mind','amind','uasum'],1)

/Users/yanfangwang/anaconda3/envs/nlp11/lib/python3.6/site-packages/ipykernel_launcher.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  if __name__ == '__main__':


Genre Preparation: 
1. find the intersection of artist between the train and test
2. treat the compound artist seperately from single artist
3. if a single artist from test set hasn't appeared before, we can do nothing, however, if a compound artist has appeared, we can possibly estimate it from the single artist's contribution
4. add a dummy variable genre_estimated to show

In [75]:
train.genre_ids = train.genre_ids.fillna('-1')
test.genre_ids = test.genre_ids.fillna('-1')

In [76]:
train_genre = set()
for genre in train.genre_ids:
    train_genre.add(genre)
    if not pd.isnull(genre) and "|" in genre:
        for sub in genre.split("|"):
            train_genre.add(sub)
test_genre_diff = set()  #只有五个单个genre没有
for name in test.genre_ids.unique():
    if not pd.isnull(name) and not "|" in name and not name in train_genre:
        test_genre_diff.add(name)
    if not pd.isnull(name) and "|" in name:
        indic = [x in train_genre for x in name.split("|")]
        if not any(indic):
            test_genre_diff.add(name)

In [77]:
train_gsame = set()
for genre in test.genre_ids.unique():
    if not pd.isnull(genre) and "|" in genre and genre in train_genre:
        train_gsame.add(genre)

In [78]:
train_gdump = set()
for genre in train.genre_ids.unique():
    if not pd.isnull(genre) and "|" in genre and not genre in train_gsame:
        train_gdump.add(genre)

In [79]:
train_gsingle = [x for x in train.genre_ids.unique() if not pd.isnull(x) and not "|" in x]
#test_newsingle = [x for x in test.genre_ids.unique() if not pd.isnull(x) and not "|" in x and not x in train_gsingle and x in train_genre]
test_newsingle = set()
for x in test.genre_ids.unique():
    if not pd.isnull(x) and "|" in x:
        indc = [sub for sub in x.split("|") if sub in train_gsingle]
        if len(indc) == 0:
            test_newsingle.add(sub)

In [80]:
test_compnew = set()
for genre in test.genre_ids.unique():
    if not pd.isnull(genre) and "|" in genre and not genre in train_genre:
        for sub in genre.split("|"):
            if not sub in train_gsingle and sub in train_genre:
                test_compnew.add(genre) 
#sub= 275/864/857

In [81]:
#prepare user,genre dictionary
genredict = dict(zip(list(train_genre),range(len(train_genre))))
ugmat = sparse.lil_matrix((len(userdict),2*len(genredict)))
v = train.groupby(['msno','genre_ids'])['target'].agg(['sum','count'])
v = v.reset_index()
v.columns.values[2:] = ['ugsum','ugcount']
v['mind'] = v.msno.apply(lambda x: userdict[x])
v['gmind'] = v.genre_ids.apply(lambda x: genredict[x])
for i in v.index:
    ugmat[v.mind[i],2*v.gmind[i]] = v.ugsum[i]
    ugmat[v.mind[i],2*v.gmind[i]+1] = v.ugcount[i]
ugmat = sparse.csc_matrix(ugmat)
for dump in train.genre_ids.unique():
    subp = [dumpart for dumpart in dump.split("|") if dumpart not in train_gsingle]
    for sub in subp:
        ugmat[:,2*genredict[sub]] += ugmat[:,2*genredict[dump]]
        ugmat[:,2*genredict[sub]+1] += ugmat[:,2*genredict[dump]+1]

/Users/yanfangwang/anaconda3/envs/nlp11/lib/python3.6/site-packages/scipy/sparse/compressed.py:774: SparseEfficiencyWarning: Changing the sparsity structure of a csc_matrix is expensive. lil_matrix is more efficient.
  SparseEfficiencyWarning)


In [82]:
#prepare the train features on genre
ogenre = train.groupby('genre_ids')['target'].agg(['sum','count'])
ogenre = ogenre.reset_index()
ogenre.columns.values[1:3] = ['genre_sum','genre_count']
train.genre_ids = train.genre_ids.astype('str')
ogenre.genre_ids = ogenre.genre_ids.astype('str')
train = train.merge(ogenre,on='genre_ids',how = 'left')
train['genre_freq'] = (train.genre_sum-train.target+1)/(train.genre_count+1)
traindump = train[train.genre_ids.isin(list(train_gdump))]
train['is_genre_dump'] = 0
ugsum = ugmat.sum(axis = 0)
for name,group in traindump.groupby("genre_ids"):
    subind = np.array([genredict[sub] for sub in name.split("|")])
    gasum = ugsum[0,2*subind].sum()/len(subind)
    gacount = ugsum[0,2*subind+1].sum()/len(subind)
    for ind in group.index:
        train.set_value(ind,'genre_sum',gasum)
        train.set_value(ind,'genre_count',gacount)
        train.set_value(ind,'genre_freq',gasum/gacount)
        train.set_value(ind,'is_genre_dump',1)
train = train.drop('genre_sum',1)
train.genre_count = train.genre_count -1 

In [83]:
#prepare the test features on genre
ogenre['genre_freq'] = ogenre.genre_sum/ogenre.genre_count
test = test.merge(ogenre,on='genre_ids',how = 'left')
test['is_genre_dump'] = 0
testnan = test[pd.isnull(test.genre_sum)]
for name, group in testnan.groupby("genre_ids"):
    if not "|" in name:
        for ind in group.index:
            test.set_value(ind,'genre_count',0)
            test.set_value(ind,'genre_freq',0.5)
    else:
        subind = np.array([genredict[sub] for sub in name.split("|") if sub in train_genre])
        gasum = ugsum[0,2*subind].sum()/len(subind)
        gacount = ugsum[0,2*subind+1].sum()/len(subind)
        for ind in group.index:
            test.set_value(ind,'genre_count',gacount)
            test.set_value(ind,'genre_freq',gasum/gacount)
            test.set_value(ind,'is_genre_dump',1)
test = test.drop('genre_sum',1)

In [84]:
#prepare the train feature on genre/user pair
train = train.merge(v,on = ['msno','genre_ids'],how = 'left')
train['ugfreq'] = (train['ugsum'] - train['target']+1)/(train['ugcount']+1)
train['ugcount'] = train['ugcount'] -1
for name,group in train[train.genre_ids.isin(list(train_gdump))].groupby('genre_ids'):
    subcolin = np.array([genredict[sub.strip()] for sub in name.split("|")])
    vsum = ugmat[:,2*subcolin][group.mind,:].sum(axis =1)
    vcount = ugmat[:,2*subcolin+1][group.mind,:].sum(axis = 1)
    gi = group.index ; gs = group.shape[0]
    for i in range(gs):
        train.set_value(gi[i],'ugfreq',vsum[i]/vcount[i] if vcount[i] > 0 else 0.5)
        train.set_value(gi[i],'ugcount',vcount[i]/len(subcolin))
train = train.drop(['mind','gmind','ugsum'],1)

In [85]:
#prepare the test feature on genre/user pair
trainmsno = train.msno.unique()
v['ugfreq'] = v.ugsum/v.ugcount
test = test.merge(v,on = ['msno','genre_ids'],how = 'left')
testnan = test[pd.isnull(test.ugcount)]
testnu = testnan[~testnan.msno.isin(trainmsno)]
for name,group in testnu.groupby(['msno','genre_ids']):
    subcolin = np.array([genredict[sub.strip()] for sub in name[1].split("|") if sub in train_genre])
    overallfreq = ugsum[0,2*subcolin].sum()/ugsum[0,2*subcolin+1].sum()
    indic = len(subcolin) > 0
    for ind in group.index:
        test.set_value(ind,'ugcount',0)
        test.set_value(ind,'ugfreq',overallfreq if indic else 0.5)
testrem = test[pd.isnull(test.ugfreq)]
testrem['mind']= testrem.msno.apply(lambda x: userdict[x])
for name,group in testrem.groupby('genre_ids'):
    subcolin = np.array([genredict[sub.strip()] for sub in name.split("|") if sub in train_genre])
    if len(subcolin) > 0:
        vsum = ugmat[:,2*subcolin][group.mind,:].sum(axis = 1)
        vcount = ugmat[:,2*subcolin+1][group.mind,:].sum(axis =1)
        gi = group.index
        for i in range(group.shape[0]):
            test.set_value(gi[i],'ugfreq',vsum[i]/vcount[i] if vcount[i] > 0 else 0.5)
            test.set_value(gi[i],'ugcount',vcount[i]/len(subcolin))
    else:
        for ind in group.index:
            test.set_value(ind,'ugcount',0)
            test.set_value(ind,'ugfreq',group.user_freq[ind])
test = test.drop(['mind','gmind','ugsum'],1)

/Users/yanfangwang/anaconda3/envs/nlp11/lib/python3.6/site-packages/ipykernel_launcher.py:15: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  from ipykernel import kernelapp as app


#compute the cosine similarity based songs score

In [11]:
from sklearn.metrics.pairwise import cosine_similarity
rowin = train['msno'].unique()
colin = train['song_id'].unique()
spm = sparse.lil_matrix((len(rowin),len(colin)))
dictuser = dict(zip(rowin,range(len(rowin))))
dictsong = dict(zip(colin,range(len(colin))))
for ind in train.index:
    userp = dictuser[train.msno[ind]]
    songp = dictsong[train.song_id[ind]]
    spm[userp,songp] = (1 if train.target[ind] == 1 else -1)
u_simi_song= cosine_similarity(spm)
spm = sparse.csr_matrix(spm)

In [ ]:
train['usbs'] = 0.0
train['usbs_is_estimate'] = 0
train['uind'] = train['msno'].apply(lambda x: dictuser[x])
train['rating'] = train['target'].apply(lambda x: 1 if x == 1 else -1)
train_gr = train.groupby('song_id')
for name,group in train_gr.groupby('song_id'):
    num = group.shape[0]
    if num > 1:
        for ind in group.index:
            sims = u_simi_song[group.uind[ind],group.uind]
            ra = (np.dot(sims,group.rating)-group.rating[ind])/(num-1)
            train.set_value(ind,'usbs',ra)
    else:
        train.set_value(group.index[0],'usbs',group.uafreq)
        train.set_value(group.index[0],'usbs_is_estimate',1)

In [6]:
#test usbs
test['usbs'] = 0.0
test['usbs_is_estimate'] = 0
for ind in test[~test.song_id.isin(train.song_id.unique()) & ~test.msno.isin(train.msno.unique())].index:
    test.set_value(ind,'usbs',0.5)
    test.set_value(ind,'usbs_is_estimate',1)
for ind in test[~test.song_id.isin(train.song_id.unique()) & test.msno.isin(train.msno.unique())].index:
    test.set_value(ind,'usbs',test.uafreq[ind])
    test.set_value(ind,'usbs_is_estimate',1)
for ind in test[test.song_id.isin(train.song_id.unique()) & ~test.msno.isin(train.msno.unique())].index:
    test.set_value(ind,'usbs',test.song_freq[ind])
    test.set_value(ind,'usbs_is_estimate',1)
for name,group in test[test.song_id.isin(train.song_id.unique()) & test.msno.isin(train.msno.unique())].groupby('song_id'):
    trainsong = train[train.song_id == name]
    for ind in group.index:
        sims = u_simi_song[group.uind[ind],trainsong.uind]
        ra = np.dot(sims,group.rating)/trainsong.shape[0]
        train.set_value(ind,'usbs',ra)

In [ ]:
train = train.drop(['uind','rating'],1)

In [13]:
#member clean
#How many days has they been active
members['rtime'] = members['registration_init_time'].apply(
    lambda x : pd.to_datetime(str(x), format='%Y%m%d'))
members['etime'] = members['expiration_date'].apply(
    lambda x : pd.to_datetime(str(x), format='%Y%m%d'))
members['numactivedays'] = (members['etime'] - members['rtime']).dt.days
min_day = members['rtime'].min()
members['day_id_res'] = (members['rtime'] - min_day).dt.days+1
members['day_id_exp'] = (members['etime'] - min_day).dt.days+1
members['r_year'] = members['rtime'].dt.year
members['r_month'] = members['rtime'].dt.month
members['r_day'] = members['rtime'].dt.day
members['e_year'] = members['etime'].dt.year
members['e_month'] = members['etime'].dt.month
members['e_day'] = members['etime'].dt.day

In [14]:
#cleaned age and cleaned_gender
members['is_male'] = members.gender == 'male'
members['is_female'] = members.gender == 'female'
members['age_mtrue'] = members.bd.apply(lambda x: 1 if x > 10 and x < 80 else 0)
weighted_age = round(members.bd[members.age_mtrue == 1].mean())
members['age_clean'] = members.bd.apply(lambda x: x if x >10 and x < 80 else weighted_age)

In [ ]:
train = train.merge(members,on = 'msno', how = 'left')
test = test.merge(members,on = 'msno', how = 'left')

In [17]:
def isrc_to_year(isrc):
    if type(isrc) == str:
        if int(isrc[5:7]) > 17:
            return 1900 + int(isrc[5:7])
        else:
            return 2000 + int(isrc[5:7])
    else:
        return np.nan

In [20]:
songs_extra = pd.read_csv('song_extra_info.csv')
songs_extra['song_year'] = songs_extra['isrc'].apply(isrc_to_year)
songs_extra.drop(['isrc', 'name'], axis = 1, inplace = True)

train = train.merge(songs_extra, on='song_id', how='left')
test = test.merge(songs_extra, on='song_id', how='left')

In [16]:
#spm processed by svd
#u,s,v = sparse.linalg.svds(spm,k = 120)

Index(['msno', 'city', 'bd', 'gender', 'registered_via',
       'registration_init_time', 'expiration_date', 'rtime', 'etime',
       'numactivedays', 'day_id_res', 'day_id_exp', 'r_year', 'r_month',
       'r_day', 'e_year', 'e_month', 'e_day', 'is_male', 'is_female',
       'age_mtrue', 'age_clean'],
      dtype='object')